# Determines mask for given sulcus

This notebook determines mask around a sulcus. The mask is built in order to only keep sulcus of interest. <br>
It uses a supervised database, in which each sulcus has been manually labelled.

# Imports

In [1]:
import sys
import os
import glob
import json
import tempfile
import inspect
import zipfile
import dico_toolbox as dtx
import colorado as cld
import pcpm

from urllib.request import urlopen
from soma import aims

In [2]:
import anatomist.notebook as ana
a = ana.Anatomist()
print(a.headless_info.__dict__)

global modules: /casa/host/build/share/anatomist-5.1/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module anacontrolmenu
loading module foldsplit
loading module simple_controls
Anatomist started.
loading module histogram
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
all python modules loaded
Anatomist started.
{'xvfb': <subprocess.Popen object at 0x7f9068311668>, 'original_display': ':1', 'display': 23, 'glx': 2, 'virtualgl': None, 'headless': True, 'mesa': False, 'qtapp': 'QtGui'}


The following line permits to import deep_folding even if this notebook is executed from the notebooks subfolder (and no install has been launched):

 /notebooks/use_transform.ipynb  
 /deep_folding/__init__.py

In [3]:
import deep_folding
print(inspect.getfile(deep_folding))

/neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/deep_folding/__init__.py


# User-specific variables

In [4]:
sulcus = 'S.T.s.ter.asc.ant.'

In [5]:
side = 'L'

In [6]:
out_voxel_size = 1

We now assign path names and other user-specific variables.

The source directory is where the database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [7]:
src_dir = os.path.join(os.getcwd(), '../data/source/supervised')
src_dir = os.path.abspath(src_dir)
print(("src_dir = " + src_dir))

src_dir = /neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/data/source/supervised


In [8]:
mask_dir = os.path.join(os.getcwd(), '../data/target/mask')
mask_dir = os.path.abspath(mask_dir)
print(("mask_dir = " + mask_dir))

mask_dir = /neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/data/target/mask


In [9]:
ref_dir = os.path.join(os.getcwd(), '../data/reference/mask')
ref_dir = os.path.abspath(ref_dir)
print(("ref_dir = " + ref_dir))

ref_dir = /neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/data/reference/mask


In [10]:
print((sys.argv))

['/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9003', '--control=9001', '--hb=9000', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"4728efc0-5174-4a99-84d5-90e18285706b"', '--shell=9002', '--transport="tcp"', '--iopub=9004', '--f=/tmp/tmp-33192UnZm23aAvQd5.json']


Gets the normlized SPM file to get voxel size inside the program

norm_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
norm_dir = os.path.abspath(norm_dir)
sub_dir = "ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition"

# Illustration of main program uses

### Using external calls

In [11]:
!python ../deep_folding/brainvisa/compute_mask.py --help

usage: compute_mask.py [-h] [-s SRC_DIR [SRC_DIR ...]] [-m MASK_DIR]
                       [-u SULCUS] [-i SIDE] [-p PATH_TO_GRAPH]
                       [-n NB_SUBJECTS] [-v] [-x OUT_VOXEL_SIZE]

Computes mask around the named sulcus

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR [SRC_DIR ...], --src_dir SRC_DIR [SRC_DIR ...]
                        Source directory where the MRI data lies. If there are
                        several directories, add all directories one after the
                        other. Example: -s DIR_1 DIR_2. Default is :
                        /neurospin/dico/data/bv_databases/human/pclean/all
  -m MASK_DIR, --mask_dir MASK_DIR
                        Output directory where to store the output mask files.
                        Default is :
                        /neurospin/dico/data/deep_folding/test/mask
  -u SULCUS, --sulcus SULCUS
                        Sulcus name around which we determine the bounding
 

### By using the main function call

In [12]:
from deep_folding.brainvisa import compute_mask
print(compute_mask.__file__)

/neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/deep_folding/brainvisa/compute_mask.py


In [13]:
args = "--help"
argv = args.split(' ')

In [14]:
compute_mask.main(argv)

usage: compute_mask.py [-h] [-s SRC_DIR [SRC_DIR ...]] [-m MASK_DIR]
                       [-u SULCUS] [-i SIDE] [-p PATH_TO_GRAPH]
                       [-n NB_SUBJECTS] [-v] [-x OUT_VOXEL_SIZE]

Computes mask around the named sulcus

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR [SRC_DIR ...], --src_dir SRC_DIR [SRC_DIR ...]
                        Source directory where the MRI data lies. If there are
                        several directories, add all directories one after the
                        other. Example: -s DIR_1 DIR_2. Default is :
                        /neurospin/dico/data/bv_databases/human/pclean/all
  -m MASK_DIR, --mask_dir MASK_DIR
                        Output directory where to store the output mask files.
                        Default is :
                        /neurospin/dico/data/deep_folding/test/mask
  -u SULCUS, --sulcus SULCUS
                        Sulcus name around which we determine the bounding
 

### By using the API function call

In [15]:
compute_mask.compute_mask(src_dir=src_dir,
                          mask_dir=mask_dir,
                          sulcus=sulcus,
                          side=side,
                          out_voxel_size=out_voxel_size,
                          number_subjects=0)


# Test example

In [16]:
unsupervised_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
reference_dir = os.path.join(os.getcwd(), '../data/reference')

In [17]:
vol_mask = compute_mask.compute_mask(src_dir=src_dir,
                                     mask_dir=mask_dir,
                                     sulcus=sulcus,
                                     side=side,
                                     number_subjects=1,
                                     out_voxel_size=1)


INFO:compute_mask.py: {'subject': 'sujet01', 'side': 'L', 'dir': '/neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/data/source/supervised', 'graph_file': 'sujet01/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Lsujet01*.arg'}
INFO:compute_mask.py: Final mask file: /neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/data/target/mask/L/S.T.s.ter.asc.ant._left.nii.gz


In [18]:
print(vol_mask.shape) 
assert(vol_mask.shape==(193, 229, 193, 1))

(193, 229, 193, 1)


In [19]:
a_vol_mask = a.toAObject(vol_mask)
axial0 = a.createWindow("Axial")
axial0.addObjects(a_vol_mask)

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

In [20]:
temp_dir = tempfile.mkdtemp()
mask_filename_temp = f"{temp_dir}/mask.nii.gz"
aims.write(vol_mask, mask_filename_temp)
bucket_filename = f"{temp_dir}/mask.bck"
cmd = f"AimsFileConvert -c Bucket -t VOID -e 1 -i {mask_filename_temp} -o {bucket_filename}"
os.system(cmd)

# Displays bucket file
bucket, bucket_raw, dxyz, rot, tr = pcpm.load_bucket(bucket_filename)
m = dtx.convert.bucket_to_mesh(bucket)
cld.draw(m)


In [21]:
cld.draw(bucket_raw)

# Mask test with more than 1 subject

This takes a supervised database outside the deep_folding/data folder

In [22]:
print(mask_dir)

/neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/data/target/mask


In [23]:
src_dir = "/neurospin/dico/data/bv_databases/human/pclean/all"
mask_dir_temp = f"{temp_dir}/mask"

vol_mask = compute_mask.compute_mask(src_dir=src_dir,
                                     mask_dir=mask_dir_temp,
                                     sulcus=sulcus,
                                     side=side,
                                     number_subjects=10,
                                     out_voxel_size=out_voxel_size)


INFO:compute_mask.py: {'subject': 'sujet01', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'sujet01/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Lsujet01*.arg'}
INFO:compute_mask.py: {'subject': 's12590', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 's12590/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Ls12590*.arg'}
INFO:compute_mask.py: {'subject': 'ammon', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'ammon/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Lammon*.arg'}
INFO:compute_mask.py: {'subject': 'vayu', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'vayu/t1mri/t1/default_analysis/folds/3.3/base2018_manual/Lvayu*.arg'}
INFO:compute_mask.py: {'subject': 'osiris', 'side': 'L', 'dir': '/neurospin/dico/data/bv_databases/human/pclean/all', 'graph_file': 'osiris/t1mri/t1/default_analysis/folds/3.3/ba

In [24]:
c = aims.Converter_rc_ptr_Volume_S16_BucketMap_VOID()
bucket = c(vol_mask)

In [25]:
# Displays bucket file
m = dtx.convert.bucket_to_mesh(bucket[0])
cld.draw(m)

We now represent the mask together with the MNI template:

In [26]:
# We recover the MNI template
install_dir = "."
extracted_dir = f"{install_dir}/mni_icbm152_nlin_asym_09c"
if os.path.exists(extracted_dir):
    print(f'the directory {extracted_dir} already exists. Assuming it is OK.')
else:
    dl_url = "http://www.bic.mni.mcgill.ca/~vfonov/icbm/2009/mni_icbm152_nlin_asym_09c_nifti.zip"
    tmp_dl = tempfile.mkstemp(suffix='.zip')
    with urlopen(dl_url) as f:
        with open(tmp_dl[1], 'wb') as g:
            g.write(f.read())
    # Extract the archive
    with zipfile.ZipFile(tmp_dl[1], 'r') as zf:
        zf.extractall(install_dir)

the directory ./mni_icbm152_nlin_asym_09c already exists. Assuming it is OK.


In [27]:
mni_file = f"{extracted_dir}/mni_icbm152_t1_tal_nlin_asym_09c.nii"
mni = a.loadObject(mni_file)

In [28]:
mask_vol_aims = vol_mask
print(mask_vol_aims.header())

{ 'volume_dimension' : [ 193, 229, 193, 1 ], 'sizeX' : 193, 'sizeY' : 229, 'sizeZ' : 193, 'sizeT' : 1, 'referential' : '84b1989b-eb68-8665-0049-8feaf3c22679', 'referentials' : [ 'Talairach-MNI template-SPM' ], 'transformations' : [ [ -1, 0, 0, 96, 0, -1, 0, 96, 0, 0, -1, 114, 0, 0, 0, 1 ] ], 'voxel_size' : [ 1, 1, 1, 1 ] }


In [29]:
# fusion 2D
mask_vol = a.toAObject(mask_vol_aims)
fusion2d = a.fusionObjects([mni, mask_vol], "Fusion2DMethod")
axial = a.createWindow("Axial")
axial.addObjects(fusion2d)
# params of the fusion : linear on non null
a.execute("Fusion2DParams", object=fusion2d, mode="linear_on_defined", rate=0.4)

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

# Result analysis

We here compare the results with a reference result made with one subject taken from the data unsupervised source folder

Prints the list of files of the target directory

In [30]:
mask_dir_side = os.path.join(mask_dir, side)
print(mask_dir_side)
print(('\n'.join(os.listdir(mask_dir_side))))

/neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/data/target/mask/L
S.T.s.ter.asc.ant._left.nii.gz.minf
S.T.s.ter.asc.ant._left.nii.gz


In [31]:
ref_mask_dir_side = os.path.join(ref_dir, side)
print(ref_mask_dir_side)
list_files = '\n'.join(os.listdir(ref_mask_dir_side))
print(list_files)

/neurospin/dico/jchavas/Runs/32_deep_folding_foldlabel_clean/Program/deep_folding/data/reference/mask/L
S.T.s.ter.asc.ant._left.nii.gz.minf
S.T.s.ter.asc.ant._left.nii.gz


In [32]:
mask_vol_ref = aims.read(glob.glob(f"{ref_mask_dir_side}/*.nii.gz")[0])

In [33]:
# fusion 2D
mask_ref = a.toAObject(mask_vol_ref)
fusion2d = a.fusionObjects([mask_ref, mask_vol], "Fusion2DMethod")
axial = a.createWindow("Axial")
axial.addObjects(fusion2d)
# params of the fusion : linear on non null
a.execute("Fusion2DParams", object=fusion2d, mode="linear_on_defined", rate=0.4)

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)